# Install Dependencies

In [1]:
!pip3 install requests playwright beautifulsoup4

In [2]:
!playwright install

# Scrape Pinterest for Images

In [3]:
url = "https://www.pinterest.com"
slug = "/pin/438889926200478583/"
depth = 2
concurrent = 10

In [4]:
#@markdown Download main image from Pinterest page
import requests
from bs4 import BeautifulSoup

def download_image(url, name):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    images = soup.find_all('img')

    main_image = images[0]
    src = main_image.get('src')
    response = requests.get(src)
    with open(f"data/pinterest/{name}.jpg", "wb") as f:
        f.write(response.content)

In [5]:
#@markdown Get recommended images from a Pinterest page
import asyncio
from playwright.async_api import async_playwright

async def get_recommended(link):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto(link)

        # Scroll down to load more images
        for _ in range(2): # One minute
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await asyncio.sleep(5)

        suggestions = await page.query_selector_all('a')
        links = []
        for suggestion in suggestions:
            href = await suggestion.get_attribute('href')
            if '/pin/' in href:
                links.append(href)
        
        await browser.close()

        print(f"Found {len(links)} pins")
        return links

In [6]:
async def find_recommended(link, depth, seen=None):
    if seen is None:
        seen = set()
        
    if depth == 0 or link in seen:
        return set()
    
    seen.add(link)
    recommended = await get_recommended(url + link)
    all_recommended = set(recommended)
    
    if depth > 1:
        tasks = [find_recommended(pin, depth - 1, seen) for pin in recommended if pin not in seen]
        results = await asyncio.gather(*tasks)
        for result in results:
            all_recommended.update(result)
    
    print(f"Found {len(all_recommended)} total pins")
    return all_recommended

In [83]:
recommended = await find_recommended(slug, depth)
for i, pin in enumerate(recommended):
    download_image(url + pin, str(i))

Found 46 pins
Found 18 pins
Found 18 pins
Found 18 pins
Found 20 pins
Found 18 pins
Found 16 pins
Found 18 pins
Found 20 pins
Found 20 pins
Found 16 pins
Found 20 pins
Found 20 pins
Found 20 pins
Found 22 pins
Found 20 pins
Found 20 pins


Error: Unable to adopt element handle from a different document

Found 24 pins
Found 22 pins
Found 24 pins
Found 24 pins
Found 24 pins
Found 24 pins
Found 24 pins
Found 0 pins
Found 0 pins
Found 0 total pins
Found 0 total pins
Found 0 pins
Found 0 total pins
Found 0 pins
Found 0 total pins
Found 44 pins
Found 22 total pins
Found 46 pins
Found 23 total pins
Found 46 pins
Found 23 total pins
